In [9]:
import wk_db

wk_db.read_sql('select count(*) from us_market_daily', db_loc='low_freq_db')

,count(*)
0,32616023


In [10]:
# idx_map = {
#     # 'spx': "SPX.GI",
#     # 'ndx': "NDX.GI"
#     # # "000001.SH":"上证综指",
#     # "000016.SH":"上证50",
#     # 'csi300': "000300.SH" # :"沪深300",
#     # "399905.SZ":"中证500",
#     # "000852.SH":"中证1000",
#     # "399006.SZ":"创业板指"
# }

In [11]:
idx_map = {
    # 'spx': "SPX.GI",
    # 'ndx': "NDX.GI"
    # # "000001.SH":"上证综指",
    # "000016.SH":"上证50",
    'csi300': "000300.SH", # :"沪深300",
    'csi500': "000905.SH", # :"中证500",
    'csi1000': "000852.SH", #:"中证1000",
    # "399006.SZ":"创业板指"
}

In [12]:

idx_df = wk_db.read_sql(f"SELECT * FROM index_components_weight;", db_loc='low_freq_db')
idx_df

,trade_dt,index_windcode,con_windcode,industry_name,weight
0,2010-01-11,000016.SH,600000.SH,银行(中信),4.110
1,2010-01-11,000016.SH,600005.SH,钢铁(中信),0.920
2,2010-01-11,000016.SH,600015.SH,银行(中信),1.350
3,2010-01-11,000016.SH,600016.SH,银行(中信),5.550
4,2010-01-11,000016.SH,600018.SH,交通运输(中信),0.890
...,...,...,...,...,...
877301,2023-11-30,000001.SH,600135.SH,基础化工(中信),0.010
877302,2023-11-30,000001.SH,600137.SH,纺织服装(中信),0.004
877303,2023-11-30,000001.SH,600138.SH,消费者服务(中信),0.017
877304,2023-11-30,000001.SH,600141.SH,基础化工(中信),0.046


In [13]:
idx_df = wk_db.read_sql(f"SELECT DISTINCT index_windcode FROM index_components_weight;", db_loc='low_freq_db')
idx_df

,index_windcode
0,000016.SH
1,399006.SZ
2,000001.SH
3,000905.SH
4,000300.SH
5,000852.SH


In [14]:
idx_name = 'csi500'
idx_code = idx_map[idx_name]

idx_df = wk_db.read_sql(f"SELECT con_windcode, MIN(trade_dt) as st, MAX(trade_dt) as ed FROM index_components_weight WHERE index_windcode='{idx_code}' GROUP BY con_windcode ORDER BY con_windcode;", db_loc='low_freq_db')
idx_df

,con_windcode,st,ed
0,000005.SZ,2010-01-04,2011-12-30
1,000006.SZ,2010-01-04,2020-05-29
2,000008.SZ,2016-06-30,2021-05-31
3,000009.SZ,2017-12-29,2024-11-29
4,000012.SZ,2014-12-31,2023-11-30
...,...,...,...
1530,688777.SH,2022-06-30,2024-11-29
1531,688778.SH,2023-06-30,2024-11-29
1532,688779.SH,2022-12-30,2024-11-29
1533,688819.SH,2022-06-30,2024-11-29


In [15]:
idx_df['con_windcode'] = idx_df['con_windcode'].apply(lambda s: s.split('.')[1] +  s.split('.')[0])
idx_df.to_csv(f'/data/linq/.qlib/qlib_data/cn_data/instruments/{idx_name}.txt', index=False, header=False, sep='\t')

In [16]:
import pandas as pd

filter_date = '2024-07-01'
filter_date = pd.to_datetime(filter_date)
filter_date = filter_date.date()
idx_df = idx_df[idx_df['ed'] > filter_date]
idx_df['st'] = '1900-01-01'
idx_df['ed'] = '2099-01-01'
idx_df['con_windcode'] = idx_df['con_windcode'].apply(lambda s: s.split('.')[1] +  s.split('.')[0])
idx_df

/tmp/ipykernel_2277357/3544429022.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  idx_df['st'] = '1900-01-01'
/tmp/ipykernel_2277357/3544429022.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  idx_df['ed'] = '2099-01-01'


IndexError: list index out of range

In [9]:
idx_df.to_csv(f'/data/linq/.qlib/qlib_data/cn_data/instruments/{idx_name}_ext.txt', index=False, header=False, sep='\t')